Note of things to fix: 
- Address extractor doesn't work if words do not have spaces between them

In [38]:
#testing uploading pdf
import PyPDF2 
import io
import pyap

def extract_pdf_content(content: bytes) -> str:
    try:
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(content))
        # pdf_reader = PyPDF2.PdfReader(content)
        text = ""
        for i, page in enumerate(pdf_reader.pages):
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print('error with file')
        #adlflafj
        raise

Testing Reading & Removing Senstiv e info from pdf file

In [ ]:

pdf_file = open("../bank_statement_BofA.pdf", "rb")
pdf_content= extract_pdf_content(pdf_file.read())

#get bank
bank= input("What is your bank?")
#removing account number(s)
num_of_accounts= input("How many account numbers do you have?")
all_account_nums=[]
for i in num_of_accounts:
    if bank=="Bank of America":
        all_account_nums.append(input("What is you account number. Please provide a space between every 4 numbers."))
    else:
        all_account_nums.append(input("What is you account number."))

#remove all of the account numbers
print(all_account_nums)
for num in all_account_nums:
    pdf_content= pdf_content.replace(num, "")


#removing address from pdf content
all_address= str(pyap.parse(pdf_content, country='US'))[1:-1]
all_address= all_address.replace(",", "")
for address_icon in all_address.split(" "):
    pdf_content= pdf_content.replace(address_icon, "")

# #removing account number(s)
# num_of_accounts= input("How many account numbers do you have?")
# all_account_nums=[]
# for i in num_of_accounts:
#     all_account_nums.append(input("What is your account number?"))

# for num in all_account_nums:
#     all_address= all_address.replace(num, "")


print(pdf_content)

Creating Read & Removing Info as a Function

In [41]:
#creating above read & redact code into a function
def read_redact_file(file_name, all_account_nums):
    pdf_content= extract_pdf_content(pdf_file.read())

    #remove all of the account numbers
    #print(all_account_nums)
    for num in all_account_nums:
        pdf_content= pdf_content.replace(num, "")

    #removing address from pdf content
    all_address= str(pyap.parse(pdf_content, country='US'))[1:-1]
    # if all_address == "":
    #     all_address = input("Type you address")
    all_address= all_address.replace(",", "")
    for address_icon in all_address.split(" "):
        pdf_content= pdf_content.replace(address_icon, "")
    return (all_address, pdf_content)

In [42]:
#code for reading all statements in statement folder
import os 
folder = '../statements'
files = os.listdir(folder)

outputs = []

#get bank
bank= input("What is your bank?")

#removing account number(s)
num_of_accounts= int(input("How many account numbers do you have?"))
all_account_nums=[]

for i in range(num_of_accounts):
    if bank=="Bank of America":
        all_account_nums.append(input("What is you account number. Please provide a space between every 4 numbers."))
    else:
        all_account_nums.append(input("What is you account number."))

#goes through each file
for statement in files: 
    pdf_file = open(f'../statements/{statement}', "rb")
    outputs.append(read_redact_file(pdf_file, all_account_nums))

In [43]:
outputs

[('4820 18th pl se everett WA',
  'MercyO.Dada 000000\n148thPlSE\nEverett,98208-8812\nPage1of2StatementPeriod:07/27/2024-08/23/2024 3224495\nSummaryofDepositAccountActivity\nAccount#Beginning\nBalanceWithdrawals/\nFees* DepositsDividends/\nInterestEnding\nBalance\nMemberAdvantageSavings 3613344881 1,403.60 (239.89) 958.30 2.75 2,124.76\nMemberAdvantageChecking 3625424853 13.01 (202.48) 239.89 0.04 50.46\n*IncludingthefollowingFees StatementPeriodTotal 2024Year-to-DateTotal\nOverdraftFees 0.00 0.00\nNon-sufficientFunds(NSF)Fees 0.00 0.00\nDepositAccountActivity\nMemberAdvantageSavings-3613344881\nIfyouraccountconvertedtoMemberAdvantageduringthestatementperiod,youearnedtherateeffectivefortheMemberShareSavingsAccountuptothedate\nofconversion.\n2.14%AnnualPercentageYieldEarnedfor28dayperiod\nAverageDailyBalance:$1,690.26\nYear-to-datedividends:$23.56\nDeposits\nDate Amount TransactionDescription\n08/02/24 231.17 ExternalDepositMASSINST4100075043PP-DIRECTDEP\n08//24 231.18 ExternalDepositMA

In [26]:
string = 'hello worldhelphigreenblue'
print(string.split())

['hello', 'worldhelphigreenblue']
